# Data Warehouse Medicare National QA - Member Enrollment Yearly

Performing QA on tables in dw_staging before moving them to data_warehouse schema

## Initialization

Just loading packages that will be used and initializing connection to GP DB.

In [5]:
import pandas as pd
import sys
import psycopg2
from tqdm.notebook import tqdm
sys.path.append('H:/uth_helpers')
from db_utils import get_dsn

In [6]:
connection = psycopg2.connect(get_dsn())
connection.autocommit = True

## Member Counts and Member Months

This table is an aggregate of the member_enrollment_monthly table where enrollment data is aggregated to a yearly level.

Because of this, we cannot compare the row count of the raw tables with this table since enrollment tables from raw data sources are usually not on a yearly level. Instead, we can try to compare the Member Months (MM) and other columns to see if the aggregrate logic implemented is working as intended

In [7]:
query = '''drop table if exists qa_reporting.dw_mcrn_mbr_enrl_yearly;
create table qa_reporting.dw_mcrn_mbr_enrl_yearly
(
    data_source text,
    calendar_year int,
    table_src text,
    dw_row_count int,
    dw_uth_mbr_id_count int,
    dw_src_mbr_id_count int,
    src_mbr_count int,
    mbr_count_diff int,
    mbr_count_percentage float,
    dw_mm int,
    src_mm int,
    mm_diff int,
    mm_diff_percentage float,
    date_generated date
);
'''

with connection.cursor() as cursor:
    cursor.execute(query)

In [8]:
with connection.cursor() as cursor:
      query = '''
insert into qa_reporting.dw_mcrn_mbr_enrl_yearly
(data_source, calendar_year, table_src, dw_row_count, dw_uth_mbr_id_count, dw_src_mbr_id_count, dw_mm,date_generated)
select data_source, 
        year, 
        table_id_src, 
        count(*),
        count(distinct uth_member_id),
        count(distinct member_id_src),
        sum(total_enrolled_months),
        now()::date
  from dw_staging.mcrn_member_enrollment_yearly
 group by 1,2,3;
      '''

      cursor.execute(query)


In [9]:
with connection.cursor() as cursor:
    
      query = '''
update qa_reporting.dw_mcrn_mbr_enrl_yearly a
set src_mbr_count = b.pat_count,
    mbr_count_diff = a.dw_src_mbr_id_count - b.pat_count,
    mbr_count_percentage = abs(a.dw_src_mbr_id_count - b.pat_count) / b.pat_count
from qa_reporting.medicare_national_counts b
where data_source = 'mcrn'
and a.calendar_year = b.year
and a.table_src = 'medicare_national.' || b.table_name
;
      '''

      cursor.execute(query)

      query = '''
with enrl_months as(
select year::int, bene_id, t.month_year_id as enrolled_month
            from medicare_national.mbsf_abcd_summary a
            cross join lateral (values (a.year || '01', a.mdcr_status_code_01), (a.year || '02', a.mdcr_status_code_02),
                              (a.year || '03', a.mdcr_status_code_03), (a.year || '04', a.mdcr_status_code_04), (a.year || '05', a.mdcr_status_code_05),
                              (a.year || '06', a.mdcr_status_code_06), (a.year || '07', a.mdcr_status_code_07), (a.year || '08', a.mdcr_status_code_08),
                              (a.year || '09', a.mdcr_status_code_09), (a.year || '10', a.mdcr_status_code_10), (a.year || '11', a.mdcr_status_code_11),
                              (a.year || '12', a.mdcr_status_code_12))
            t(month_year_id, enrollment_status)
            where t.enrollment_status in ('10','11','20','21','31')
),
enrl_my as (
    select 'medicare_national.mbsf_abcd_summary' table_name, year, count(enrolled_month) as mm
    from enrl_months
    group by 2
)
update qa_reporting.dw_mcrn_mbr_enrl_yearly a
set src_mm = b.mm,
    mm_diff = a.dw_mm - b.mm,
    mm_diff_percentage = abs(a.dw_mm - b.mm) / b.mm
from enrl_my b
where data_source = 'mcrn'
and a.calendar_year = b.year
and a.table_src = b.table_name
;
      '''

      cursor.execute(query)

In [10]:
df = pd.read_sql('''select * 
from qa_reporting.dw_mcrn_mbr_enrl_yearly 
order by mm_diff ;''', con=connection)
df

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,data_source,calendar_year,table_src,dw_row_count,dw_uth_mbr_id_count,dw_src_mbr_id_count,src_mbr_count,mbr_count_diff,mbr_count_percentage,dw_mm,src_mm,mm_diff,mm_diff_percentage,date_generated
0,mcrn,2016,medicare_national.mbsf_abcd_summary,3045820,3045820,3045820,3046466,-646,0.0,34821230,34821230,0,0.0,2023-06-26
1,mcrn,2015,medicare_national.mbsf_abcd_summary,2962729,2962729,2962729,2963123,-394,0.0,33818812,33818812,0,0.0,2023-06-26
2,mcrn,2014,medicare_national.mbsf_abcd_summary,2877140,2877140,2877140,2877408,-268,0.0,32819005,32819005,0,0.0,2023-06-26
3,mcrn,2020,medicare_national.mbsf_abcd_summary,3381851,3381851,3381851,3382077,-226,0.0,38770661,38770661,0,0.0,2023-06-26
4,mcrn,2017,medicare_national.mbsf_abcd_summary,3132606,3132606,3132606,3133013,-407,0.0,35827216,35827216,0,0.0,2023-06-26
5,mcrn,2019,medicare_national.mbsf_abcd_summary,3306584,3306584,3306584,3306928,-344,0.0,37904124,37904124,0,0.0,2023-06-26
6,mcrn,2018,medicare_national.mbsf_abcd_summary,3228969,3228969,3228969,3229338,-369,0.0,36929927,36929927,0,0.0,2023-06-26


Here we look at the overall difference of member enrollment and member count. We start of by just adding the counts for each of the source tables. However, due to how the yearly table is built, it may be better to look at member counts and member months as the data source overall.

In [11]:
df.groupby('calendar_year')[['mm_diff', 'mm_diff_percentage', 'mbr_count_diff', 'mbr_count_percentage']].sum()

,mm_diff,mm_diff_percentage,mbr_count_diff,mbr_count_percentage
calendar_year,,,,
2014,0,0.0,-268,0.0
2015,0,0.0,-394,0.0
2016,0,0.0,-646,0.0
2017,0,0.0,-407,0.0
2018,0,0.0,-369,0.0
2019,0,0.0,-344,0.0
2020,0,0.0,-226,0.0


## Gender Counts

Just like with the monthly enrollment table, we need to check the counts and values of the other columns. Here we take a look at the gender_cd column

In [12]:
query = '''with mcrn_gen_cd as (
    select distinct year::int, bene_id, sex_ident_cd
    from medicare_national.mbsf_abcd_summary a
),
mcrn_gen as (
    select year, c.gender_cd, count(*) gender_count
    from mcrn_gen_cd m
    left join reference_tables.ref_gender c
    on c.data_source = 'mcr'
   and c.gender_cd_src = m.sex_ident_cd
    group by 1,2
), dw_gen as (
    select year, gender_cd, count(*) gender_count
    from dw_staging.mcrn_member_enrollment_yearly
    group by 1,2
)
select a.*, b.gender_count as src_gender_count, 
        a.gender_count - b.gender_count as gender_diff, 
        100. * abs(a.gender_count - b.gender_count) / b.gender_count as gender_diff_percent
from mcrn_gen b
full outer join dw_gen a
on a.year = b.year
and a.gender_cd = b.gender_cd
order by year, gender_cd;
'''

pd.read_sql(query,  con=connection)

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,year,gender_cd,gender_count,src_gender_count,gender_diff,gender_diff_percent
0,2014,F,1574156,1574291,-135,0.008575
1,2014,M,1302984,1303117,-133,0.010206
2,2015,F,1620802,1621011,-209,0.012893
3,2015,M,1341927,1342112,-185,0.013784
4,2016,F,1666383,1666729,-346,0.020759
5,2016,M,1379437,1379737,-300,0.021743
6,2017,F,1713826,1714080,-254,0.014818
7,2017,M,1418780,1418933,-153,0.010783
8,2018,F,1768342,1768547,-205,0.011591
9,2018,M,1460627,1460791,-164,0.011227


## Plan Type

Check if plan types are properly mapped at a yearly level. If the plan type counts do not match between the member_enrollment_yearly table and the raw enrollment table, it most likely occurred when cleaning up the plan_type column. Ideally the count difference percent should be <= 1%.

In [13]:
query = '''with mcrn_enroll as (
    select year::int, bene_id, ent.plan_type, t.month_year_id
    from medicare_national.mbsf_abcd_summary a
    cross join lateral (values (year || '01', a.mdcr_entlmt_buyin_ind_01, a.mdcr_status_code_01), (year || '02', a.mdcr_entlmt_buyin_ind_02, a.mdcr_status_code_02),
                        (year || '03', a.mdcr_entlmt_buyin_ind_03, a.mdcr_status_code_03), (year || '04', a.mdcr_entlmt_buyin_ind_04, a.mdcr_status_code_04), (year || '05', a.mdcr_entlmt_buyin_ind_05, a.mdcr_status_code_05),
                        (year || '06', a.mdcr_entlmt_buyin_ind_06, a.mdcr_status_code_06), (year || '07', a.mdcr_entlmt_buyin_ind_07, a.mdcr_status_code_07), (year || '08', a.mdcr_entlmt_buyin_ind_08, a.mdcr_status_code_08),
                        (year || '09', a.mdcr_entlmt_buyin_ind_09, a.mdcr_status_code_09), (year || '10', a.mdcr_entlmt_buyin_ind_10, a.mdcr_status_code_10), (year || '11', a.mdcr_entlmt_buyin_ind_11, a.mdcr_status_code_11),
                        (year || '12', a.mdcr_entlmt_buyin_ind_12, a.mdcr_status_code_12))
    t(month_year_id, mcdcr_enrlmt, enrollment_status)
    join reference_tables.ref_medicare_entlmt_buyin ent 
    on ent.buyin_cd = t.mcdcr_enrlmt
    where t.enrollment_status in ('10','11','20','21','31')
),
mcrn_agg_enroll as (
    select year, bene_id, plan_type, max(month_year_id) my, count(*)
    from mcrn_enroll
    group by 1,2,3
),
mcrn_grp_enroll as (
    select *, row_number() over(partition by bene_id, year order by count desc, my desc) as my_grp
    from mcrn_agg_enroll
),
mcrn_plans as (
    select year, plan_type, count(*) plan_count
    from mcrn_grp_enroll 
    where my_grp = 1
    group by 1,2
),
dw_plans as (
    select year, plan_type, count(*) plan_count
    from dw_staging.mcrn_member_enrollment_yearly
    group by 1,2
)
select a.*, b.plan_count as src_plan_count, 
        a.plan_count - b.plan_count as plan_diff, 
        100. * abs(a.plan_count - b.plan_count) / b.plan_count as plan_diff_percent
from mcrn_plans b
full outer join dw_plans a
on a.year = b.year
and a.plan_type = b.plan_type
order by year;
'''

plan_type_counts_df = pd.read_sql(query,  con=connection)
plan_type_counts_df

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,year,plan_type,plan_count,src_plan_count,plan_diff,plan_diff_percent
0,2014,AB,1747217,2598087.0,-850870.0,32.749866
1,2014,A,261218,260337.0,881.0,0.338408
2,2014,B,17452,18716.0,-1264.0,6.753580
3,2014,C,851253,NaN,NaN,NaN
4,2015,B,17616,18880.0,-1264.0,6.694915
5,2015,A,269348,268472.0,876.0,0.326291
6,2015,C,917651,NaN,NaN,NaN
7,2015,AB,1758114,2675377.0,-917263.0,34.285374
8,2016,B,17832,18757.0,-925.0,4.931492
9,2016,AB,1781833,2748167.0,-966334.0,35.162856


In [14]:
plan_type_counts_df.groupby('year')['plan_diff_percent'].min(), plan_type_counts_df.groupby('year')['plan_diff_percent'].max()

(year
 2014    0.338408
 2015    0.326291
 2016    0.313378
 2017    0.332327
 2018    0.349295
 2019    0.178243
 2020    0.502133
 Name: plan_diff_percent, dtype: float64,
 year
 2014    32.749866
 2015    34.285374
 2016    35.162856
 2017    36.985815
 2018    38.899472
 2019    40.877648
 2020    43.698926
 Name: plan_diff_percent, dtype: float64)

In [15]:
plan_type_counts_df.sort_values('plan_diff_percent')

,year,plan_type,plan_count,src_plan_count,plan_diff,plan_diff_percent
23,2019,A,311365,310811.0,554.0,0.178243
10,2016,A,279770,278896.0,874.0,0.313378
5,2015,A,269348,268472.0,876.0,0.326291
13,2017,A,291342,290377.0,965.0,0.332327
1,2014,A,261218,260337.0,881.0,0.338408
16,2018,A,304241,303182.0,1059.0,0.349295
27,2020,A,307231,305696.0,1535.0,0.502133
8,2016,B,17832,18757.0,-925.0,4.931492
15,2017,B,17790,18779.0,-989.0,5.266521
4,2015,B,17616,18880.0,-1264.0,6.694915


In [16]:
plan_types_agg_df = plan_type_counts_df.groupby('year')['plan_count', 'src_plan_count', 'plan_diff'].sum()
plan_types_agg_df['percent_diff'] = 100. * plan_types_agg_df['plan_diff'] / plan_types_agg_df['src_plan_count'] 
plan_types_agg_df

C:\Users\iperez9\AppData\Local\Temp\20\ipykernel_16644\1871058508.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  plan_types_agg_df = plan_type_counts_df.groupby('year')['plan_count', 'src_plan_count', 'plan_diff'].sum()


,plan_count,src_plan_count,plan_diff,percent_diff
year,,,,
2014,2877140,2877140.0,-851253.0,-29.586777
2015,2962729,2962729.0,-917651.0,-30.973167
2016,3045820,3045820.0,-966385.0,-31.728237
2017,3132606,3132606.0,-1044300.0,-33.336462
2018,3228969,3228969.0,-1130919.0,-35.024152
2019,3306584,3306584.0,-1217527.0,-36.821294
2020,3381851,3381851.0,-1335948.0,-39.503455


## State

In [17]:
query = '''with mcrn_enroll as (
    select year::int, bene_id, case when state_cd is null then 'XX' else state_cd end as state
    from medicare_national.mbsf_abcd_summary m
    left outer join reference_tables.ref_medicare_state_codes e 
     on e.medicare_state_cd = m.state_code   
),
mcrn_states as (
    select year, state, count(*) state_count
    from mcrn_enroll m
    group by 1,2
), 
dw_plans as (
    select year, case when state is null then 'XX' else state end as state, count(*) state_count
    from dw_staging.mcrn_member_enrollment_yearly
    group by 1,2
)
select a.*, b.state_count as src_state_count, 
        a.state_count - b.state_count as plan_diff, 
        100. * abs(a.state_count - b.state_count) / b.state_count as plan_diff_percent
from mcrn_states b
full join dw_plans a
on a.year = b.year
and a.state = b.state
order by year;
'''

df = pd.read_sql(query,  con=connection)
df

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,year,state,state_count,src_state_count,plan_diff,plan_diff_percent
0,2014,MS,28830,28831,-1,0.003468
1,2014,PA,133029,133036,-7,0.005262
2,2014,FL,205690,205702,-12,0.005834
3,2014,NV,23178,23179,-1,0.004314
4,2014,IL,108233,108235,-2,0.001848
...,...,...,...,...,...,...
373,2020,TX,230797,230803,-6,0.002600
374,2020,PA,149655,149657,-2,0.001336
375,2020,AL,56695,56700,-5,0.008818
376,2020,AK,5595,5595,0,0.000000


We see that there is a large difference in state values assigned using DW logic compared to raw data. Ideally the difference should show that there are more values in the raw data compared to the values in DW due, especially for state values of 'XX'. When this happens, it means that the member now has a valid state value which can be used to group these member in future projects.

In [18]:
df.sort_values('plan_diff_percent', ascending=False)

,year,state,state_count,src_state_count,plan_diff,plan_diff_percent
135,2016,XX,24690,24916,-226,0.907048
28,2014,XX,24196,24248,-52,0.214451
274,2019,XX,26878,26915,-37,0.137470
252,2018,XX,26188,26219,-31,0.118235
102,2015,XX,24182,24205,-23,0.095022
...,...,...,...,...,...,...
158,2016,NH,14669,14669,0,0.000000
294,2019,DC,5138,5138,0,0.000000
171,2017,NM,21322,21322,0,0.000000
172,2017,NE,17988,17988,0,0.000000


Should get 50 states plus Puerto Rico (PR), Washington D.C. (DC), Virgin Islands (VI), and unknown (XX)

In [19]:
df['state'].unique()

array(['MS', 'PA', 'FL', 'NV', 'IL', 'MT', 'MD', 'ND', 'VA', 'HI', 'NE',
       'DE', 'NC', 'GA', 'CO', 'MN', 'ME', 'WA', 'LA', 'NH', 'DC', 'PR',
       'TX', 'NY', 'SD', 'SC', 'TN', 'AZ', 'XX', 'MI', 'NM', 'CT', 'MO',
       'OR', 'AL', 'OH', 'KY', 'VT', 'RI', 'WY', 'MA', 'IA', 'WV', 'CA',
       'UT', 'AK', 'KS', 'IN', 'ID', 'OK', 'WI', 'VI', 'NJ', 'AR'],
      dtype=object)

In [20]:
df['state'].unique().shape

(54,)

In [21]:
df[df['state'] == '']

,year,state,state_count,src_state_count,plan_diff,plan_diff_percent


In [22]:
df[df['state'].isna()]

,year,state,state_count,src_state_count,plan_diff,plan_diff_percent
